# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Data import

In [ ]:
df = pd.read_csv('./data/model_data.csv')

# Convert date column to datetime and sort chronologically
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date').reset_index(drop=True)

df

## Data splitting

In [ ]:
# Define input and output features.
# Note: 'date' is not used directly in the regression model.
input_features = [
    'precipitation + irrigation (mm)',
    'potential evapotranspiration (mm)',
    # 'year',
    # 'month',
    # 'day',
    # 'day_of_week',
    # 'day_of_year',
    'precip_pet_diff',
    'precip_7d_avg',
    'pet_7d_avg',
    # 'precip_7d_sum',
    # 'pet_7d_sum'
]

output_features = [
    'depth 10cm',
    'depth 30cm',
    'depth 60cm',
    'depth 90cm',
    'actual evapotranspiration (mm)',
    'groundwater recharge (mm)'
]

# Perform a time-based train/test split (80% train, 20% test)
split_index = int(len(df) * 0.8)
train = df.iloc[:split_index]
test = df.iloc[split_index:]

X_train = train[input_features]
y_train = train[output_features]
X_test = test[input_features]
y_test = test[output_features]
y = df[output_features]

In [ ]:
y.describe()

## Baseline results

In [ ]:
# baseline
y_pred_baseline = np.full(y_test.shape, y_train.mean())

baseline_mse = mean_squared_error(y_test, y_pred_baseline)
print(f'Baseline MSE: {baseline_mse:.2f}')

# Models

## Multi-output linear regression

In [ ]:
# Train a multi-output linear regression model
model = MultiOutputRegressor(LinearRegression())
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse_per_target = mean_squared_error(y_test, y_pred, multioutput='raw_values')
overall_mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error for each target variable:")
for col, mse in zip(output_features, mse_per_target):
    print(f"  {col}: {mse:.2f}")
print(f"\nOverall Mean Squared Error: {overall_mse:.2f}")


In [ ]:
# Plot the actual vs. predicted values for each target variable
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (col, ax) in enumerate(zip(output_features, axes)):
    ax.plot(test['date'], y_test[col], label='Actual', color='blue')
    ax.plot(test['date'], y_pred[:, i], label='Predicted', color='red')
    ax.set_title(col)
    ax.legend()

plt.tight_layout()
plt.show()

# Cross-validation - (todo)

In [ ]:
# df = pd.read_csv('./data/model_data.csv')

# # Convert date column to datetime and sort chronologically
# df['date'] = pd.to_datetime(df['date'])
# df = df.sort_values(by='date').reset_index(drop=True)

# # Define input and output features.
# # 'date' is only used for sorting and plotting.
# input_features = [
#     'precipitation + irrigation (mm)',
#     'potential evapotranspiration (mm)',
#     # 'year',
#     # 'month',
#     # 'day',
#     # 'day_of_week',
#     # 'day_of_year',
#     'precip_pet_diff',
#     'precip_7d_avg',
#     'pet_7d_avg',
#     # 'precip_7d_sum',
#     # 'pet_7d_sum'
# ]

# output_features = [
#     'depth 10cm',
#     'depth 30cm',
#     'depth 60cm',
#     'depth 90cm',
#     'actual evapotranspiration (mm)',
#     'groundwater recharge (mm)'
# ]

# # Prepare the input (X) and output (y) DataFrames
# X = df[input_features]
# y = df[output_features]

# # Time Series Cross-Validation using TimeSeriesSplit
# tscv = TimeSeriesSplit(n_splits=5)
# cv_mse_scores = []

# for fold, (train_index, test_index) in enumerate(tscv.split(X)):
#     X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
#     y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]
    
#     model_cv = MultiOutputRegressor(LinearRegression())
#     model_cv.fit(X_train_cv, y_train_cv)
#     y_pred_cv = model_cv.predict(X_test_cv)
    
#     mse_cv = mean_squared_error(y_test_cv, y_pred_cv)
#     cv_mse_scores.append(mse_cv)
#     print(f"Fold {fold+1} - MSE: {mse_cv:.2f}")

# print("\nTime Series Cross-Validation Results:")
# print("MSE Scores for each fold:", cv_mse_scores)
# print("Average MSE across folds:", np.mean(cv_mse_scores))